In [5]:
import sys
sys.path.insert(0, '../')
from config import *
from dataset import *
import umap
import umap.plot
from trainers.adv_g import *
from trainers.trainer import *

## OoD Generation via Adversarial Attack

### Load MNIST dataset

In [12]:
B = 128
val_B = 64

In [13]:
# mnist_tri_set, mnist_val_set, mnist_tri_loader, mnist_val_loader = MNIST(B, 32, 2, True)
# cifar_tri_set, cifar_val_set , cifar_tri_loader, cifar_val_loader = CIFAR10(B, 32)
# USE MNIST SUBSAMPLES
ind_idx = [0,2,3,6,8]
ood_idx = [1,7]
# TODO: Use classifier on 1,4,5,7,9, check wass loss
# test_idx = [4,5,9]
mnist_dset_dict = MNIST_SUB(batch_size=B, val_batch_size=val_B, idx_ind=ind_idx, idx_ood=ood_idx, shuffle=True)
# TODO: Show dataset statistics and sample images.

#### Pretrain discriminator (i.e. classifier)

In [15]:
img_info = {'H':28,'W':28,'C':1}
D = dc_discriminator(img_info, GAN_TYPE.OOD).to(DEVICE)
ind_train_loader = mnist_dset_dict['train_set_ind_loader']
ind_val_loader = mnist_dset_dict['val_set_ind_loader']
train(D,ind_train_loader,ind_val_loader, num_epoch=3)

ic| f"Learning rate = {1e-4}": 'Learning rate = 0.0001'
  0%|          | 0/3 [00:00<?, ?it/s]

Epoch  # 1 | training loss: 0.5934966845765646               | training acc: 0.8551825612219137


 33%|███▎      | 1/3 [00:26<00:52, 26.39s/it]

Epoch  # 1 | validation loss: 0.13500042020892486               | validation acc: 0.9577323717948718
Epoch  # 2 | training loss: 0.11685005640970791               | training acc: 0.9635694111335522


 67%|██████▋   | 2/3 [00:52<00:26, 26.36s/it]

Epoch  # 2 | validation loss: 0.07462152342001598               | validation acc: 0.9742665187376727
Epoch  # 3 | training loss: 0.07501827731584658               | training acc: 0.9769478982580156


100%|██████████| 3/3 [01:18<00:00, 26.27s/it]

Epoch  # 3 | validation loss: 0.05083613257067135               | validation acc: 0.9822793392504932


#### Save/Load pretrained discriminator (if necessary)

In [16]:
path = '../checkpoint/adv_pre_D.pt'

In [17]:
torch.save({'model_state_dict': D.state_dict()}, path)

In [18]:
pretrain = torch.load(path)
D.load_state_dict(pretrain['model_state_dict'])
print(f"Pretrained D state is loaded from {path}.")

Pretrained D state is loaded from ../checkpoint/adv_pre_D.pt.


### Adversarial Generation

In [19]:
M = 100 # B * M is the number of samples we want
nl_Wt = 1.5 # Negative log of Wasserstein distance

In [ ]:
img = adv_generation(ind_tri_loader, D, B, M, nl_Wt)
torch.save(img, '../checkpoint/adv_g_img.pt')
# img: BM x 1 x 28 x 28 for MNIST

### UMAP Visualization

In [ ]:
img = torch.load('../checkpoint/adv_g_img.pt')

#### In-Distribution vs. Generated OoD (Verbose)

In [20]:
x_ind = mnist_dset_dict['train_set_ind']
x_ind_y = torch.tensor([x[1] for x in x_ind]) # All y labels
x_ind = torch.stack([x[0].flatten() for x in x_ind]) # All images
y_ind = torch.ones((len(x_ind))) # All ones for in-distribution samples
ic(x_ind.shape)
ic(x_ind_y.shape)

ic| x_ind.shape: torch.Size([29781, 784])
ic| x_ind_y.shape: torch.Size([29781])


torch.Size([29781])

In [ ]:
x_g = img.flatten(1,3)
y_g = torch.ones(x_g.shape[0], dtype=torch.int) * -1
ic(x_g.shape)
ic(y_g.shape)

In [21]:
x_ood = mnist_dset_dict['train_set_ood']
x_ood_y = torch.tensor([x[1] for x in x_ood])
x_ood = torch.stack([x[0].flatten() for x in x_ood])
ic(x_ood.shape)
ic(x_ood_y.shape)

ic| x_ood.shape: torch.Size([13007, 784])
ic| x_ood_y.shape: torch.Size([13007])


torch.Size([13007])

In [ ]:
x_vis = torch.cat((x_ood, x_g, x_ind),dim=0)
y_vis = torch.cat((x_ood_y, y_g, x_ind_y), dim=0)
ic(x_vis.shape)
ic(y_vis.shape)

In [ ]:
umap_visualization(x_vis, y_vis)